<h1>Utility Methods for Bitcoin Network and Bitcoin Blockchain</h1>

<h2>Structure of Block</h2>
<h4>Block Header</h4>
<p>
[4 Bytes (Mainnet: F9 BE B4 D9 or 0xD9B4BEF9, Testnet: FA BF B5 DA or 0xDAB5BFFA)] Magic ID<br>
[4 Bytes] Block length<br>
[4 Bytes] Version<br>
[32 Bytes] Previous Block Hash<br>
[32 Bytes] Merkle Tree Root<br>
[4 Bytes] Timestamp<br>
[4 Bytes] Bits<br>
[4 Bytes] Nounce<br>
[less than 0xfd is 1 byte, 0xfd is 2 bytes, 0xfe is 4 bytes, 0xff is 8 bytes] Size of Number of Transactions<br>
[1-8 Bytes] Number of Transactions<br>
</p>

<h4>P2PKH Transaction Format With Example</h4>
<p>
01 00 00 00 version (1)<br>
01 number of inputs<br>
7f 95 0a b7 90 83 8e 0c 05 e7 98 56 d2 5d 58 68 23 fe 13 9e 18 07 40 5a 3f 20 7f f3 3f 9b 76 63 previous tx hash<br>
01 00 00 00<br>
6b script length (107 bytes)<br>
48 sig length (72 bytes)<br>
30 DER<br>
45 length of sig ECDSA (69)<br>
02 int type<br>
21 length of R (33 bytes)<br>
00 d8 62 94 03 cd 3b 49 95 0d a9 29 36 53 c6 27 91 49 c0 29 e6 b7 b1 53 71 34 2d 0d 2c e2 86 c8 f2 (R)<br>
02 int type<br>
20 length of S (32 bytes)<br>
78 78 79 85 a6 44 e9 4f d9 24 6f 6c 25 73 33 36 c9 4a f5 f0 0d 9d 34 a0 7d c2 f9 e0 98 7e f9 90<br>
01 SIGHASH_ALL<br>
21 length of pubkey (33 bytes)<br>
02 public key type (0x2 is compressed pubkey(x) with even y, 0x3 is compressed pubkey(x) with odd y, 0x4 full pubkey)<br>
b7 26 d7 ea e1 1a 6d 5c f3 b2 36 2e 77 3e 11 6a 61 40 34 7d ce e1 b2 94 3f 4a 28 97 35 1e 5d 90 (pubkey)<br>
ff ff ff ff Sequence<br>
02 number of outputs<br>
1b f0 3c 00 00 00 00 00 Value (3993627 santoshi)<br>
17 lenght of scriptPubKey (23 bytes)<br>
a9 OP_HASH160<br>
14 Hash size (20 which is 160 bits)<br>
69 f3 75 73 80 a5 68 20 ab c7 05 28 67 21 65 99 e5 75 cd dd Hash160<br>
87 OP_EQUAL<br>
77 c1 ca 1c 00 00 00 00 Value ()<br>
19 length of scriptPubKey (25 bytes)<br>
76 OP_DUP<br>
a9 OP_HASH160<br>
14 Hash size (20)<br>
d5 f9 50 ab e0 b5 59 b2 b7 a7 ab 3d 18 a5 07 ea 1c 3e 4a c6 Hash160<br>
88 OP_EQUALVERIFY<br>
ac OP_CHECKSIG<br>
00 00 00 00 Lock time<br>
</p>
<h2>Building important Methods and Constants</h2>
<p> Values in Bitcoin network is in Little Endian while most tools accepts and returns Big Endian values. This is taken care in below program</p>
<br>

In [8]:
import hashlib
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
import simplejson as json
import binascii
import base58
import ecdsa
from pycoin.ecdsa.numbertheory import modular_sqrt
import pycoin
import pandas as pd
import sys

# Add correct Bitcoin Address
myaddress = 'xxxxxxxxxxxxxxxxxxxxxxxxx'

rpc_connection = AuthServiceProxy("http://%s:%s@127.0.0.1:8332"%('alice', 'passw0rd'))

SANTOSIS_IN_BTC = 10**8

DIFFICULTY_1 = 0x00ffff * (256 ** 26)

# Target Difficulty set every 2016 blocks
TARGET_DIFFICULTY_SET_EVERY = 2016

# Every Block Height at which Block Reward becomes Half
BLOCK_REWARD_HALVING = 210000
BLOCK_REWARD_1 = 50 * SANTOSIS_IN_BTC

def hash2LittleEndian2LittleEndian(a:str, b:str):
     # Reverse inputs before and after hashing due to big-endian / little-endian nonsense
     a1 = binascii.unhexlify(a)[::-1]
     b1 = binascii.unhexlify(b)[::-1]
     h = hashlib.sha256(hashlib.sha256(a1 + b1).digest()).digest()
     return binascii.hexlify(h[::-1])

def hashBigEndian2LittleEndian(a: str):
     h = hashlib.sha256(hashlib.sha256(bytes.fromhex(a)).digest()).digest()
     return binascii.hexlify(h[::-1])

def double_sha256d(bstr):
    return hashlib.sha256(hashlib.sha256(bstr).digest()).digest()


<h2>Finding Blockchain Related Information</h2>
<br>

In [9]:
def getCurrentBlockHeight():
    current_block_height = rpc_connection.getblockcount()
    return current_block_height

In [10]:
def getCurrentBlockReward():
    block_height = getCurrentBlockHeight()
    block_halving_count = int(block_height / BLOCK_REWARD_HALVING)
    current_block_reward = BLOCK_REWARD_1 / (2 ** block_halving_count)
    return current_block_reward

In [7]:
def getCurrentBitcoinInCirculation():
    block_height = getCurrentBlockHeight()
    block_halving_count = int(block_height / BLOCK_REWARD_HALVING)
    block_reward = BLOCK_REWARD_1 / SANTOSIS_IN_BTC
    bitcoin_in_circulation = 0
    for block_halfing_index in range(block_halving_count):
        bitcoin_in_circulation += (BLOCK_REWARD_HALVING * block_reward)
        block_reward = block_reward / 2
    bitcoin_in_circulation += (block_height % BLOCK_REWARD_HALVING) * block_reward
    return bitcoin_in_circulation